# Project: Cleaning Data and Hypothesis Testing

First, let's get to know some definitions:

* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

The Hypothesis we will test is:

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

You can look for the data in this project on the rest of the repository, the data files are from:

* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. Look at GDP data from the first quarter of 2000 onward.

Let's import the following libraries:

In [7]:
import pandas as pd
import numpy as np
from scipy import stats

## Get list of university towns

For the first part I'm going to return a DataFrame of towns and the states they are in from the 
university_towns.txt list.

The following cleaning was done:
1. For "State", removing characters from "[" to the end
2. For "RegionName", when applicable, removing every character from " (" to the end.
3. Depending on how you read the data, you may need to remove newline character '\n'. 

In [1]:
def get_list_of_university_towns():
    Utowns=pd.read_csv('university_towns.txt',sep='\n',header=None,names=['Towns'])
    indices=[]
    for i in range(0,len(Utowns['Towns'])):
        if 'edit' in Utowns.iloc[i,0]:
            indices.append(i)
    Utowns1=Utowns.drop(indices)
    numerorepeticiones=indices+[len(Utowns)]
    nombres=Utowns.iloc[indices,:]
    repeticiones=[]
    for i in range(1,len(numerorepeticiones)):
        rep=numerorepeticiones[i]-numerorepeticiones[i-1]-1
        repeticiones=repeticiones+[nombres.iloc[i-1,0]]*rep
    Utowns1.insert(0,'State',repeticiones)
    Utowns1=Utowns1.rename(columns={'Towns':'RegionName'})
    for i in range(0,len(Utowns1['State'])):
        if '[' in Utowns1.iloc[i,0]:
            numero=Utowns1.iloc[i,0].find('[')
            Utowns1.iloc[i,0]=Utowns1.iloc[i,0][:numero]
    for i in range(0,len(Utowns1['RegionName'])):
        if '(' in Utowns1.iloc[i,1]:
            numero=Utowns1.iloc[i,1].find('(')
            numero=numero-1
            Utowns1.iloc[i,1]=Utowns1.iloc[i,1][:numero]
    Utowns1=Utowns1.reset_index(drop=True)
    return Utowns1
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


## Get recession start

Now I'm going to return the year and quarter of the recession start time as a 
string value in a format such as 2005q3

In [2]:
def get_recession_start():
    GDP=pd.read_excel('gdplev.xls',skiprows=5,usecols=[4,6])
    GDP.columns=['Quarter','gdp2009']
    GDP=GDP.dropna()
    GDP=GDP[GDP['Quarter']>='2000q1']
    GDP=GDP.reset_index(drop=True)

    for i in range(0,len(GDP['gdp2009'])-2):
        if (GDP.iloc[i,1]>=GDP.iloc[i+1,1] and GDP.iloc[i+1,1]>=GDP.iloc[i+2,1]):
            start=i+1
            break
    
    return GDP.iloc[start,0]
get_recession_start()

'2008q3'

## Get recession end

I will return now the year and quarter of the recession end time as a 
string value in a format such as 2005q3

In [3]:
def get_recession_end():
    GDP=pd.read_excel('gdplev.xls',skiprows=5,usecols=[4,6])
    GDP.columns=['Quarter','gdp2009']
    GDP=GDP.dropna()
    GDP=GDP[GDP['Quarter']>='2000q1']
    GDP=GDP.reset_index(drop=True)
    
    for i in range(0,len(GDP['gdp2009'])-2):
        if (GDP.iloc[i,1]>=GDP.iloc[i+1,1] and GDP.iloc[i+1,1]>=GDP.iloc[i+2,1]):
            start=i+1
            break

    for i in range(start,len(GDP['gdp2009'])-2):
        if (GDP.iloc[i,1]<=GDP.iloc[i+1,1] and GDP.iloc[i+1,1]<=GDP.iloc[i+2,1]):
            end=i+2
            break
    return GDP.iloc[end,0]
get_recession_end()

'2009q4'

## Get recession bottom

Now I'm going to return the year and quarter of the recession bottom time as a 
string value in a format such as 2005q3

In [4]:
def get_recession_bottom():
    GDP=pd.read_excel('gdplev.xls',skiprows=5,usecols=[4,6])
    GDP.columns=['Quarter','gdp2009']
    GDP=GDP.dropna()
    GDP=GDP[GDP['Quarter']>='2000q1']
    GDP=GDP.reset_index(drop=True)
    
    for i in range(0,len(GDP['gdp2009'])-2):
        if (GDP.iloc[i,1]>=GDP.iloc[i+1,1] and GDP.iloc[i+1,1]>=GDP.iloc[i+2,1]):
            start=i+1
            break

    for i in range(start,len(GDP['gdp2009'])-2):
        if (GDP.iloc[i,1]<=GDP.iloc[i+1,1] and GDP.iloc[i+1,1]<=GDP.iloc[i+2,1]):
            end=i+2
            break
    bottom=GDP.iloc[start:end,:]
    minimo=bottom['gdp2009'].min()
    bottom1=bottom[bottom['gdp2009']==minimo]
    return bottom1.iloc[0,0]
get_recession_bottom()

'2009q2'

## Convert housing data to quarters

In this part, I will convert the housing data to quarters and return it as mean 
values in a dataframe. This dataframe should be a dataframe with
columns for 2000q1 through 2016q3, and should have a multi-index
in the shape of ["State","RegionName"]

In [5]:
def convert_housing_data_to_quarters():
    houses=pd.read_csv('City_Zhvi_AllHomes.csv')
    houses.head()
    num=houses.columns.get_loc('2000-01')
    years=houses.iloc[:,num:]
    n=0
    numeros=list(range(1,68))#list range mas facil
    final1=pd.DataFrame()
    for i in numeros:
        if i!=67:
            Trimestre='Trimestre'+str(i)
            quarters=years.iloc[:,n:n+3].mean(axis=1)
            quarters=quarters.rename(Trimestre)
            final1=pd.concat([final1,quarters],axis=1)
            n=n+3
        else:
            Trimestre='Trimestre'+str(i)
            quarters=years.iloc[:,n:n+2].mean(axis=1)
            quarters=quarters.rename(Trimestre)
            final1=pd.concat([final1,quarters],axis=1)
    GDP=pd.read_excel('gdplev.xls',skiprows=5,usecols=[4,6])
    GDP.columns=['Quarter','gdp2009']
    GDP=GDP.dropna()
    GDP=GDP[GDP['Quarter']>='2000q1']
    GDP=GDP.reset_index(drop=True)
    nombres=[]
    for i in range(0,len(GDP['Quarter'])):
        nombres.append(GDP.iloc[i,0])
    nombres.append('2016q3')
    final1.columns=nombres
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    final1.insert(0,'state',houses['State'])
    final1.insert(1,'RegionName',houses['RegionName'])
    final1.set_index('state',inplace=True)
    final1.insert(0,'State',pd.Series(states))
    final1=final1.reset_index(drop=True)
    final1.set_index(['State','RegionName'],inplace=True)
    return final1
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


## Run the hypothesis test

For the last part, I will create new data showing the decline or growth of housing prices
between the recession start and the recession bottom. Then runs a ttest
comparing the university town values to the non-university towns values, 
return whether the alternative hypothesis (that the two groups are the same)
is true or not as well as the p-value of the confidence.

I will return a tuple (different, p, better) where different=True if the t-test is
True at a p<0.01 (we reject the null hypothesis), or different=False if 
otherwise (we cannot reject the null hypothesis). The variable p should
be equal to the exact p value returned from scipy.stats.ttest_ind(). The
value for better should be either "university town" or "non-university town"
depending on which has a lower mean price ratio (which is equivilent to a
reduced market loss).

In [10]:
def run_ttest():
    data=convert_housing_data_to_quarters()
    data1=get_list_of_university_towns()

    final=pd.merge(data,data1,how='inner',left_index=True, right_on=['State','RegionName'])
    final=final.reset_index(drop=True)
    final.set_index(['State','RegionName'],inplace=True)
    final['Ratio']=final['2008q3']/final['2009q2']
    final1=data.merge(data1,how='left',left_index=True, right_on=['State','RegionName'],indicator=True).query('_merge=="left_only"').drop('_merge',1)
    final1=final1.reset_index(drop=True)
    final1.set_index(['State','RegionName'],inplace=True)
    final1['Ratio1']=final1['2008q3']/final1['2009q2']
    prueba=stats.ttest_ind(final['Ratio'], final1['Ratio1'],nan_policy='omit')
    if prueba[1]<0.01:
        different=True
        better= "university town"
    else:
        different=False
        better="non-university town"
    return (different,prueba[1],better)
run_ttest()

(True, 0.005496427353633026, 'university town')